https://github.com/PacktPublishing/Python-Social-Media-Analytics/blob/5886b29587054526920e40b54ea630de5ed9d365/Chapter03/Chapter3.ipynb


https://developers.facebook.com/docs/graph-api/explorer/

https://developers.facebook.com/docs/places/search

https://developers.facebook.com/docs/graph-api/reference/page/ratings/

https://developers.facebook.com/tools/explorer/

In [7]:
#!pip install facepy
#!pip install pony

In [8]:
import requests
import json
import time
import db_reacao
from datetime import datetime
from facepy import GraphAPI
from pony import orm

In [9]:
def insert_facebook(cliente, page_id, query_time_post, query_time_comm, get_feed,access_token=False):
    global erros_insert_comment
    global insert_comment
    min_datetime_target = False
    
    post={}
    comm={}
    reac={}
    reac_sentimento = {"SAD":-1,"ANGRY":-1,"HAHA":1,"LOVE":1,"WOW":1,"LIKE":1}
    
    if get_feed==True:
        
        if access_token==False:
            str_url_facebook = 'v2.12/' + page_id + '/feed?'
        else:
            str_url_facebook = 'v2.12/' + page_id.split("v2.12/")[1].split("/feed?access_token=")[0] + '/feed?access_token=' + page_id.split("v2.12/")[1].split("/feed?access_token=")[1]
        
        data_post = graph.get(str_url_facebook)
        for p in range(0,len(data_post["data"])):
            post = data_post["data"][p]
            created_time_post = datetime.strptime(post.get("created_time").split("T")[0], '%Y-%m-%d')
            if created_time_post>=query_time_post:
                data_comm = graph.get('v2.12/'+ post.get("id") +'?fields=comments,reactions')
                if data_comm.get("comments")!=None:                
                    for c in range(0,len(data_comm["comments"]["data"])):
                        comm = data_comm["comments"]["data"][c]
                        created_time_comm = datetime.strptime(comm.get("created_time").split("T")[0], '%Y-%m-%d')
                        if created_time_comm>=query_time_comm:
                            try:
                                insert_facebook(cliente, comm.get("id"), query_time_post, query_time_comm,False)
                                db_reacao.add_reacao_face(cliente,2,comm["message"], 0,str(created_time_comm),str(comm["id"]))
                                #db_reacao.add_reacao_face(cliente,2,comm["message"], 0,int(time.mktime(created_time_comm.timetuple())),str(comm["id"]))
                                print("Registro inserido".center(110,"*"))
                                print("Comentário: " + comm["message"])
                                insert_comment += 1
                            except Exception as e:
                                erros_insert_comment += 1
                                print("Registro com erro".center(110,"*"))
                                print("Comentário: " + comm["message"])
                                print("Erro: " + str(e))  
                if data_comm.get("reactions")!=None:
                    for c in range(0,len(data_comm["reactions"]["data"])):
                        reac = data_comm["reactions"]["data"][c]
                        try:            
                            db_reacao.add_reacao_face(cliente,3,reac["type"], reac_sentimento.get(reac["type"],0),str(datetime.now()),str(post["id"])+"_"+str(reac["id"]))
                            #db_reacao.add_reacao_face(cliente,3,reac["type"], reac_sentimento.get(reac["type"],0),int(time.mktime(datetime.now().timetuple())),str(post["id"])+"_"+str(reac["id"]))
                            print("Registro inserido".center(110,"*"))
                            print("Reação: " + reac["type"])
                            insert_comment += 1
                        except Exception as e:
                            erros_insert_comment += 1
                            print("Registro com erro".center(110,"*"))
                            print("Reação: " + reac["type"])
                            print("Erro: " + str(e))    
            else:
                min_datetime_target=True
                
        if min_datetime_target==False:
            if data_post.get('paging')!=None:
                next_page = data_post.get('paging')
                if next_page.get('next')!=None:
                    next_page_id = next_page.get('next')
                    insert_facebook(cliente, next_page_id, query_time_post, query_time_comm,get_feed,True)
            
    else:
        data_comm = graph.get('v2.12/'+ page_id +'?fields=comments,reactions')        
        if data_comm.get("comments")!=None:                
            for c in range(0,len(data_comm["comments"]["data"])):
                comm = data_comm["comments"]["data"][c]
                created_time_comm = datetime.strptime(comm.get("created_time").split("T")[0], '%Y-%m-%d')
                if created_time_comm>=query_time_comm:
                    try:
                        insert_facebook(cliente, comm.get("id"), query_time_post, query_time_comm,False)
                        db_reacao.add_reacao_face(cliente,2,comm["message"], 0,str(created_time_comm),str(comm["id"]))
                        #db_reacao.add_reacao_face(cliente,2,comm["message"], 0,int(time.mktime(created_time_comm.timetuple())),str(comm["id"]))
                        print("Registro inserido".center(110,"*"))
                        print("Comentário: " + comm["message"])
                        insert_comment += 1
                    except Exception as e:
                        erros_insert_comment += 1
                        print("Registro com erro".center(110,"*"))
                        print("Comentário: " + comm["message"])
                        print("Erro: " + str(e))  
        if data_comm.get("reactions")!=None:
            for c in range(0,len(data_comm["reactions"]["data"])):
                reac = data_comm["reactions"]["data"][c]
                try:            
                    db_reacao.add_reacao_face(cliente,3,reac["type"], reac_sentimento.get(reac["type"],0),str(datetime.now()),str(post["id"])+"_"+str(reac["id"]))
                    #db_reacao.add_reacao_face(cliente,3,reac["type"], reac_sentimento.get(reac["type"],0),int(time.mktime(datetime.now().timetuple())),str(post["id"])+"_"+str(reac["id"]))
                    print("Registro inserido".center(110,"*"))
                    print("Reação: " + reac["type"])
                    insert_comment += 1
                except Exception as e:
                    erros_insert_comment += 1
                    print("Registro com erro".center(110,"*"))
                    print("Reação: " + reac["type"])
                    print("Erro: " + str(e))    

In [10]:
# Initialize the Graph API with a valid access token (optional,
# but will allow you to do all sorts of fun stuff).
graph = GraphAPI("EAACEdEose0cBANNoFLBEohDIqEhatas5QB8vaYcbtTPNMNaFqBngi8lnZAfZCVwXO4UGenYrQ8bA2DiUS0vamvytrHO40iSJpysot3X6JMBJOzEtMYw24rERxeI5llZA6HERcrWxcgAd6AETIUkGu9E6GaM116mlsoyr2BFRM7wO8oihtGrBd2YhPvvhQkZD")

In [11]:
query_time_post = datetime.strptime("2018-04-01", '%Y-%m-%d')
query_time_comm = datetime.strptime("2018-04-01", '%Y-%m-%d')

In [12]:
erros_insert_comment = 0
insert_comment=0
get_feed = True
cliente = '1'
#page_id = '745618718900423'
page_id = 'casariobotequim'
#page_id = 'oleodealegriavv'

insert_facebook(cliente, page_id, query_time_post, query_time_comm,get_feed)

print("Resumo do processamento".center(110,"*"))
print("Total de inserções de comentario: " + str(insert_comment))
print("Erros de inserção de comentario: " + str(erros_insert_comment))

**********************************************Registro inserido***********************************************
Comentário: Tá vendo, vou fazer minha plaquinha hauahuabau
**********************************************Registro inserido***********************************************
Comentário: Jessyca Braga nigrinha
**********************************************Registro inserido***********************************************
Comentário: Pela honestidade,esse eu ajudo 🤣😂🤣😂
É vc Jessyca Braga🤔
**********************************************Registro inserido***********************************************
Comentário: Casa Rio Botequim, morada do samba. Estão todos convidados!
**********************************************Registro com erro***********************************************
Comentário: Que maravilha!!!!!!
Erro: Object Reacao[new:5] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.identificador_fonte
**********************************************Reg

In [14]:
erros_insert_comment = 0
insert_comment=0
get_feed = True
cliente = '4'
#page_id = '745618718900423'
page_id = 'gandinifabricio'
#page_id = 'oleodealegriavv'

insert_facebook(cliente, page_id, query_time_post, query_time_comm,get_feed)

print("Resumo do processamento".center(110,"*"))
print("Total de inserções de comentario: " + str(insert_comment))
print("Erros de inserção de comentario: " + str(erros_insert_comment))

**********************************************Registro inserido***********************************************
Comentário: Parabéns pela caminhada meu nobre. Sucesso na nova empreitada. O parlamento precisa de pessoas com sua capacidade e envergadura.
**********************************************Registro inserido***********************************************
Comentário: Deus abençoe meu irmão Fabrício Gandini!
**********************************************Registro inserido***********************************************
Comentário: Parabéns pelo trabalho. Que o Senhor continue te abençoando e te nutrindo. Menino de ouro!!! Exemplo a ser seguido. 💪💪💪🙌🙌🙌🙌
**********************************************Registro inserido***********************************************
Comentário: parabens gandine
**********************************************Registro inserido***********************************************
Comentário: Parabéns  pela caminhada, sucesso!
***************************************

In [15]:
erros_insert_comment = 0
insert_comment=0
get_feed = True
cliente = '5'
#page_id = '745618718900423'
page_id = 'vereadorlelecooficial'
#page_id = 'oleodealegriavv'

insert_facebook(cliente, page_id, query_time_post, query_time_comm,get_feed)

print("Resumo do processamento".center(110,"*"))
print("Total de inserções de comentario: " + str(insert_comment))
print("Erros de inserção de comentario: " + str(erros_insert_comment))

**********************************************Registro inserido***********************************************
Comentário: Feliz Páscoa Leleco
**********************************************Registro inserido***********************************************
Comentário: Feliz Páscoa amigo, para você e toda sua família. 👍
**********************************************Registro inserido***********************************************
Comentário: Feliz Páscoa ! P vc e toda sua família!
**********************************************Registro inserido***********************************************
Comentário: Feliz Páscoa amigo!
**********************************************Registro inserido***********************************************
Comentário: Feliz Páscoa pra vc e família amigo!!! 😍❤
**********************************************Registro inserido***********************************************
Comentário: Feliz Páscoa meu amigo!!!
**********************************************Registro inserid